In [1]:
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import dateutil.parser as dparser
import datetime
import re

from bs4 import BeautifulSoup
import requests

# Live Movies airing, scraped from Imdb
url='https://www.imdb.com/movies-in-theaters/'
resp=requests.get(url)
soup=BeautifulSoup(resp.text)
llist=soup.find_all('h4')
moviecounter=1
movielist=[]
for x in llist:
    for y in x.find_all('a'):
        movielist.append((moviecounter,y.text.rsplit(' ',1)[0]))
        moviecounter=moviecounter+1
error=False
yes_labels = ['y','yes','Y','Yes','YES','OK','ok','Ok','yeah','Yeah','YEAH']

#entities required to make a reservation
global theatre
global time
global date
global cd
global location
global Owner

theatre=""
cd=""
location=""
Owner=""

#to check if date exists in a given string
def is_date(string, fuzzy=False):
    global time
    global date
    """
    Return whether the string can be interpreted as a date.

    :param string: str, string to check for date
    :param fuzzy: bool, ignore unknown tokens in string if True
    """
    try: 
        dparser.parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False

#To extract the date and validate it,
def find_date(text):
    global time
    global date
    if(is_date(text,fuzzy = True)):
        x = dparser.parse(text,fuzzy = True)
        print("So movie at ",x.time(),"on ",x.date().replace(year=2019),", Can you confirm?")
        y = input()
        yes_labels = ['y','yes','Y','Yes','YES','OK','ok','Ok','yeah','Yeah','YEAH']
        words = re.split(' |, |; ',y)
        isapprove = [it in yes_labels for it in words]
        if True in isapprove:
            return x
        text = input("Please enter the date and time again")
        x = find_date(text)
        return x
    else:
        text = input("Sorry, unable to find the date and time in the text. Could you please mention the date and time again?")
        x = find_date(text)
        return x
    
    
#Function to accept sentence and extract given entities from the sentence.
def getOwner(whattoget):
    global theatre
    global cd
    global location
    global Owner
    print("Please enter "+whattoget+"\n")
    if('theatre' in whattoget):
        globalcomp='theatre'
    if('registration' in whattoget):
        globalcomp='owner'
    if('tickets' in whattoget):
        globalcomp='tickets'
    if('city' in whattoget):
        globalcomp='location'
    name=input()
    table=evaluate(name)
    tempname=""
    
    for i in range(len(table)):
        if(table[i][1]=='PERSON' and Owner==""):
            tempname+=table[i][0]
    if(tempname!=""):
        print("Can you confirm the name: ")
        print(tempname)
        print("\n")
        y = input()
        words = re.split(' |, |; ',y)
        isapprove = [it in yes_labels for it in words]
        if True in isapprove:
            print("okay")
            Owner=tempname
            
    for i in range(len(table)):
        if(table[i][1]=='LOCATION' and location==""):
            print("Can you confirm the location: ")
            print(table[i][0])
            print("\n")
            y = input()
            words = re.split(' |, |; ',y)
            isapprove = [it in yes_labels for it in words]
            if True in isapprove:
                print("okay")
                location=table[i][0]
        if(table[i][1]=='O' and table[i][2]=='CD' and cd==""):
            print("Can you confirm that you need "+table[i][0]+" tickets?")
            y = input()
            words = re.split(' |, |; ',y)
            isapprove = [it in yes_labels for it in words]
            if True in isapprove:
                print("okay")
                cd=table[i][0]
        if((i+1<(len(table)-1))and(i>-1)):
            if( ((table[i][1]=='LOCATION' and table[i+1][0]=='Theatre') or (table[i][1]=='ORGANIZATION' and table[i+1][0]=='Theatre')) and theatre==""):
                print("Is the theatre name ",table[i][0]+" "+table[i+1][0])
                print("\n")
                y = input()
                words = re.split(' |, |; ',y)
                isapprove = [it in yes_labels for it in words]
                if True in isapprove:
                    print("okay")
                    theatre=table[i][0]+" "+table[i+1][0]
    if(globalcomp=='theatre' and theatre==''):
        print("I'm sorry, i cant seem to recognize "+whattoget+" appropriately\n")
        getOwner(whattoget)
    if(globalcomp=='tickets' and cd==''):
        print("I'm sorry, i cant seem to recognize "+whattoget+" appropriately\n")
        getOwner(whattoget)
    if(globalcomp=='location' and location==''):
        print("I'm sorry, i cant seem to recognize "+whattoget+" appropriately\n")
        getOwner(whattoget)
    if(globalcomp=='owner' and Owner==''):
        print("I'm sorry, i cant seem to recognize "+whattoget+" appropriately\n")
        getOwner(whattoget)
    return

#tokenizes and classifies them using StanfordNER's 3class classifier
def evaluate(text):
    global theatre
    global time
    global date
    global cd
    global location
    global Owner
    present=False
    tokenized_text = word_tokenize(text)
    classified_text = st.tag(tokenized_text)
    table=[]
    inter=[]
    
    for i in classified_text:
        inter=[]
        inter.append(i[0])
        inter.append(i[1])
        table.append(inter)
    
    tags=pos_tag(word_tokenize(text))
    count=0
    for i in table:
        i.append(tags[count][1])
        count=count+1    
    
    print(table) #ALL THE WORDS AND THEIR NER AND POS TAGS
    return table
    
#Checks for missing entities and calls "getOwner" function for each one of them
def entervalues(text,table):
    global theatre
    global time
    global date
    global cd
    global location
    global Owner
    final=[]
    for i in range(len(table)):
        if(table[i][2]=='CD' or table[i][1]=='LOCATION' or table[i][1]=='PERSON'):
            if(table[i][1]=='LOCATION'):
                if(table[i][0]=='Theatre'):
                    if((i<(len(table)-1))and(i>-1)):
                        table[i-1][0]=table[i-1][0]+' Theatre'
                        removevalue=table[i]
            final.append(table[i])
    
    try:
        final.remove(removevalue)
    except:
        pass
#     print("\nRequired values for booking\n")
#     print(final) #THE LIST OF ALL THOSE VALUES NEED FOR A TRAVEL BOOKING

    print("\nThese are the movies airing live in theatres right now, Enter a movie number: \n\n")
    for i in movielist:
        print(str(i[0])+' '+i[1])
    
    movienumber=int(input())
    for i in movielist:
        if(i[0]==movienumber):
            movie=i[1]
    
    x = find_date(text)
    
    time=x.time()
    date=x.date()
    date=date.replace(year=2019)

    tempname=""
    for i in final:
        if(i[1]=='PERSON' and Owner==""):
            tempname+=i[0]+" "
    if(tempname!=""):
        print("Can you confirm the name: ")
        print(tempname)
        print("\n")
        y = input()
        words = re.split(' |, |; ',y)
        print(words)
        isapprove = [it in yes_labels for it in words]
        if True in isapprove:
            print("okay")
            Owner=tempname
        else:
            getOwner("the registration name")

    for i in final:
        if(i[2]=='CD' and i[1]!='TIME' and i[1]!='DATE'):
            if(':' not in i[0]):
                try:
                    if(i[0]==str(int(i[0]))):
                        print("Can you confirm that this is the number of tickets you want to book? ")
                        print(i[0])
                        print("\n")
                        y = input()
                        words = re.split(' |, |; ',y)
                        print(words)
                        isapprove = [it in yes_labels for it in words]
                        if True in isapprove:
                            print("okay")
                            cd=i[0]
                        else:
                            getOwner("the number of tickets to book")
                except:
                    pass
        elif(i[1]=='LOCATION'):
            if('Theatre'in i[0]):
                print("Can you confirm that this is the theatre you requested:\n")
                print(i[0])
                print("\n")
                y = input()
                words = re.split(' |, |; ',y)
                print(words)
                isapprove = [it in yes_labels for it in words]
                if True in isapprove:
                    print("okay")
                    theatre=theatre+i[0]
                else:
                    getOwner("the theatre name")
            else:
                print("Can you confirm the location:\n")
                print(i[0])
                print("\n")
                y = input()
                words = re.split(' |, |; ',y)
                isapprove = [it in yes_labels for it in words]
                if True in isapprove:
                    print("\nokay")
                    location=location+i[0]  
                else:
                    getOwner("the city")
    
    global error
    if(Owner==""):
        print("Whose name should i register the tickets under\n")
        getOwner("the registration name")
    if(cd==""):
        print("\nHow many of you are going for the movie?")
        getOwner("the number of tickets to book")
    if(location==""):
        print("You missed entering the location\n")
        getOwner("the city")
    if(theatre==""):
        print("You missed entering the theatre name\n")
        getOwner("the theatre name")
    #output HAS THE FULL LIST OF ALL THOSE VALUES NEEDED.
    output=dict()
    output['NUMBER OF TICKETS']=cd
    output['THEATRE NAME']=theatre
    output['MOVIE NAME']=movie
    output['SHOW TIME']=time
    output['DATE OF MOVIE']=date
    output['LOCATION']=location
    output['TICKET(S) REGISTERED UNDER']=Owner
    return(output)

st = StanfordNERTagger('/home/milan/Downloads/NLP/stanfordNER/classifiers/english.all.3class.distsim.crf.ser.gz',
					   '/home/milan/Downloads/NLP/stanfordNER/stanford-ner.jar')

print("Tell me if you are bored, i can book the tickets for a movie for you and your friends :)\n")
text = input()
temp=evaluate(text)
output=entervalues(text,temp)
print("\nYay, your tickets have been confirmed, Here are the details, enjoy the movie\n")
for i in output:
    print(i," : ",output[i])
    
#Movie Name 
#Theatre name (Done)
#Location (Done)
#Number of Tickets (Done)
#Date(Done)
#Time(Done)
#Owner

#Book 2 tickets in Pvr Theatre Bangalore at 9:00 in the morning on 7th Jan for K Varuna

Tell me if you are bored, i can book the tickets for a movie for you and your friends :)

yes
[['yes', 'O', 'NNS']]

These are the movies airing live in theatres right now, Enter a movie number: 


1  Avengers: Endgame
2  The White Crow
3  Be Natural: The Untold Story of Alice Guy-Blaché
4  Shazam!
5  Pet Sematary
6  Dumbo
7  Us
8  Captain Marvel
9  The Best of Enemies
10  Five Feet Apart
11  Unplanned
12  Wonder Park
13  How to Train Your Dragon: The Hidden World
1
Sorry, unable to find the date and time in the text. Could you please mention the date and time again?5th May at 5:00 pm
So movie at  17:00:00 on  2019-05-05 , Can you confirm?
yes
Whose name should i register the tickets under

Please enter the registration name

I would like to book 2 tickets at Pvr Theatre Bangalore for Lokesh
[['I', 'O', 'PRP'], ['would', 'O', 'MD'], ['like', 'O', 'VB'], ['to', 'O', 'TO'], ['book', 'O', 'NN'], ['2', 'O', 'CD'], ['tickets', 'O', 'NNS'], ['at', 'O', 'IN'], ['Pvr', 'O', 'NNP'], ['Theatre',